In [1]:
## import the package 
from neo4j import GraphDatabase
uri = "neo4j://localhost:7687"    ## bolt driver -- binary format for faster performance 

In [2]:
fname  = "file:///stock_data_for_graph_orig.csv"

In [3]:
## open a connection to the database 
driver = GraphDatabase.driver(uri, auth=("neo4j", "neo4j9"))
coin_fnames = ["coin_Bitcoin.csv","coin_Ethereum.csv","coin_USDCoin.csv","coin_Cardano.csv","coin_Litecoin.csv","coin_XRP.csv","coin_CryptocomCoin.csv",
             "coin_Solana.csv","stock_data_for_graph_orig.csv","coin_Dogecoin.csv","coin_Uniswap.csv"]

fname = coin_fnames[0]


In [5]:
############# Neo4j functions and queries 


### load the csv files 
def load_csv_coin_file(tx, fname): 
    ## set up the query as a string with the filename that can be modified 
    qry = (f"LOAD CSV WITH HEADERS FROM \"file:///{fname}\" as row "
           "MERGE (s:Stock{name:row.Symbol}) "
           "CREATE (s)-[:TRADING_DAY]->(:StockTradingDay{date:date(row.Date), "
           "open:toFloat(row.Open), high:toFloat(row.High), close:toFloat(row.Close), "
           "volume:toFloat(row.Volume),marketcap:toFloat(row.Marketcap)});")
    
    tx.run(qry)

### link the trading days to the stock
def link_trading_days(tx):
    qry = ("MATCH (s:Stock)-[:TRADING_DAY]->(day) "
            "WITH s, day "
            "ORDER BY day.date ASC "
            "WITH s, collect(day) as nodes, collect(day.close) as closes "
            "SET s.close_array = closes "
            "WITH nodes "
            "CALL apoc.nodes.link(nodes, 'NEXT_DAY') "
            "RETURN distinct 'done' AS result; ")
    tx.run(qry) 

### compute the pearson correlation coefficient 
def pearson_similiarity(tx): 
    qry = ("MATCH (s:Stock) "
            "WITH {item:id(s), weights: s.close_array} AS stockData "
            "WITH collect(stockData) AS input "
            "CALL gds.alpha.similarity.pearson.write({ "
            "  data: input, "
            "  topK: 3, "
            "  similarityCutoff: 0.2 "
            "}) "
            "YIELD nodes, similarityPairs "
            "RETURN nodes, similarityPairs;") 
    
    tx.run(qry) 

### determine the louvain communities 
def louvain_community(tx): 
    
    qry = ("ALL gds.louvain.write({ "
           "nodeProjection:'Stock', "
           "relationshipProjection:'SIMILAR', "
           "writeProperty:'louvain' "
           "});")
    tx.run(qry)


### add labels 
def add_labels(tx): 
    qry = ("MATCH (s:Stock)-[:TRADING_DAY]->(day) "
           "CALL apoc.create.addLabels( day, [s.name]) YIELD node "
           "RETURN distinct 'done';")
    tx.run(qry) 

### get paths 
def get_paths_stocks(tx):
    qry = ("MATCH (s:Stock)-[:TRADING_DAY]->(day) "
           "WHERE NOT ()-[:NEXT_DAY]->(day) "
           "MATCH p=(day)-[:NEXT_DAY*0..]->(next_day) "
           "SET next_day.index = length(p);") 
    tx.run(qry) 

## compute the regression 
def get_regression(tx): 
    qry = ("MATCH (s:Stock) "
           "CALL apoc.math.regr(s.name, 'close', 'index') YIELD slope "
           "SET s.slope = slope;")
    tx.run(qry) 
    
### return the community of stocks
def get_communities(tx): 
    qry = ("MATCH (s:Stock) "
           "WITH s.louvain AS community, s.slope AS slope, s.name AS ticker "
           "ORDER BY slope DESC "
           "RETURN community, collect(ticker)[..3] as potential_investments;")
    tx.run(qry) 
    
     
def delete_all_nodes(tx): 
    tx.run('''MATCH(p) 
                OPTIONAL MATCH (p)-[r]-() //drops p's relations
                DELETE r,p
            ''')
 


# def create_friend_of(tx, name, friend):
#    tx.run("MATCH (a:Person) WHERE a.name = $name "
#           "CREATE (a)-[:KNOWS]->(:Person {name: $friend})",
#           name=name, friend=friend)



In [53]:
fname = coin_fnames[0]
name="test"

qry = (f"LOAD CSV WITH HEADERS FROM \"file:///{fname}\" as row "
    "MERGE (s:Stock{name:row.Symbol}) "
    "CREATE (s)-[:TRADING_DAY]->(:StockTradingDay{date:date(row.Date), "
    "open:toFloat(row.Open), high:toFloat(row.High), close:toFloat(row.Close), "
     "volume:toFloat(row.Volume),marketcap:toFloat(row.Marketcap)});")

qry
#
#qry = (
#     f"Hi {name}. "
#     "You are a {profession}. "
#     "You were in {affiliation}."
# )

'LOAD CSV WITH HEADERS FROM "file:///coin_Bitcoin.csv" as row MERGE (s:Stock{name:row.Symbol}) CREATE (s)-[:TRADING_DAY]->(:StockTradingDay{date:date(row.Date), open:toFloat(row.Open), high:toFloat(row.High), close:toFloat(row.Close), volume:toFloat(row.Volume),marketcap:toFloat(row.Marketcap)});'

In [57]:
## load in the data 
with driver.session() as session:
    session.write_transaction(load_csv_coin_file,coin_fnames[0])

CypherSyntaxError: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input ':': expected <init> (line 1, column 1 (offset: 0))
":auto USING PERIODIC COMMIT LOAD CSV WITH HEADERS FROM "file:///{fname}" as row MERGE (s:Stock{name:row.Symbol}) CREATE (s)-[:TRADING_DAY]->(:StockTradingDay{date:date(row.Date), open:toFloat(row.Open), high:toFloat(row.High), close:toFloat(row.Close), volume:toFloat(row.Volume),marketcap:toFloat(row.Marketcap)});"
 ^}

In [ ]:
:auto USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM "file:///coin_Bitcoin.csv" as row
MERGE (s:Stock{name:row.Symbol})
CREATE (s)-[:TRADING_DAY]->(:StockTradingDay{date: date(row.Date), open:toFloat(row.Open), high:toFloat(row.High), close:toFloat(row.Close), volume: toFloat(row.Volume),marketcap: toFloat(row.Marketcap)});

In [55]:
## delete all the nodes in the database 
with driver.session() as session:
    session.write_transaction(delete_all_nodes)
    
    

In [ ]:
with driver.session() as session:
    session.write_transaction(create_friend_of, "Alice", "Bob")

with driver.session() as session:
    session.write_transaction(create_friend_of, "Alice", "Carl")


In [14]:
# close the connection to the database 
driver.close()